In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TF debugg log

# Auxiliar imports
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Model auxiliar imports
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import keras_tuner

# Model imports
from tensorflow.keras.models import load_model, Sequential  # Load saved model
from tensorflow.keras.optimizers import Adam  # Optimizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
# from tensorflow.keras.applications import Res

In [9]:
def getSetDir(set="train"):
    dirs = [f"./../dataset/archive/dataset/{set}/"]
    return dirs


# Load dataset training data
def loadData(set="train", verbose=False, size=30):
    data = []
    labels = []

    for dir in getSetDir(set):
        for subdir in os.listdir(dir):
            for img in os.listdir(dir + subdir)[0:size-1]:
                arrayLikeImage = load_img(dir + subdir + "/" + img, target_size=(300, 300))
                # arrayLikeImage = img_to_array(arrayLikeImage)
                data.append(arrayLikeImage)
                labels.append(subdir)
            if verbose:
                print(f"Done with [{subdir}] images.")

    return data, labels


trainData, trainLabels = loadData("train", size=100)
testData, testLabels = loadData("test", size=10)


In [3]:
# One-Hot Encode dataset
lb = LabelBinarizer()
trainLabels = lb.fit_transform(trainLabels)
# trainLabels = to_categorical(trainLabels)  # Turn each label into a binary array

testLabels = lb.fit_transform(testLabels)
# testLabels = to_categorical(testLabels)  # Turn each label into a binary array

trainData = np.array(trainData, dtype="float32")
trainLabels = np.array(trainLabels)
testData = np.array(testData, dtype="float32")
testLabels = np.array(testLabels)

In [4]:
model = Sequential()

# Create Model
kernelWindow = (3, 3)
# model.add(Conv2D(32, kernel_size=kernelWindow[0], activation="relu", input_shape=trainData[0].shape))
# model.add(MaxPooling2D(pool_size=kernelWindow))
model.add(Conv2D(32, kernel_size=kernelWindow[0], activation="relu"))
model.add(MaxPooling2D(pool_size=kernelWindow))
model.add(Flatten(name="flattten"))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(26, activation="softmax"))

In [16]:
# Compile the model
STEP_LEN = 1e-6
EPOCHS = 3
BATCH_SIZE = 30
opt = Adam(learning_rate=STEP_LEN, decay=STEP_LEN / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [18]:
# Train
T = model.fit(
    x=trainData,
    y=trainLabels,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(trainData) // BATCH_SIZE,
    validation_data=(testData, testLabels),
    validation_steps=len(testData) // BATCH_SIZE,
    epochs=EPOCHS)

Epoch 1/3
85/85 [==============================] - 9s 95ms/step - loss: 3.2525 - accuracy: 0.0404 - val_loss: 3.3383 - val_accuracy: 0.0381
Epoch 2/3
85/85 [==============================] - 8s 90ms/step - loss: 3.2520 - accuracy: 0.0409 - val_loss: 3.3383 - val_accuracy: 0.0381
Epoch 3/3
85/85 [==============================] - 8s 95ms/step - loss: 3.2475 - accuracy: 0.0417 - val_loss: 3.3383 - val_accuracy: 0.0381


In [14]:
print("- [INFO] evaluating network...")
predictedIdxs = model.predict(testData, batch_size = BATCH_SIZE)
predictedIdxs = np.argmax(predictedIdxs, axis=1)
print(classification_report(testLabels.argmax(axis = 1), predictedIdxs, target_names = lb.classes_))

- [INFO] evaluating network...
8/8 [==============================] - 1s 74ms/step
              precision    recall  f1-score   support

           A       0.00      0.00      0.00         9
           B       0.00      0.00      0.00         9
           C       0.00      0.00      0.00         9
           D       0.00      0.00      0.00         9
           E       0.00      0.00      0.00         9
           F       0.00      0.00      0.00         9
           G       0.00      0.00      0.00         9
           H       0.00      0.00      0.00         9
           I       0.00      0.00      0.00         9
           J       0.00      0.00      0.00         9
           K       0.00      0.00      0.00         9
           L       0.00      0.00      0.00         9
           M       0.00      0.00      0.00         9
           N       0.00      0.00      0.00         9
           O       0.00      0.00      0.00         9
           P       0.00      0.00      0.00         

/home/vinicius-verona/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vinicius-verona/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vinicius-verona/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
